# Numerical linear algebra

System information (for reproducibility)

In [1]:
versioninfo()

Julia Version 1.7.3
Commit 742b9abb4d (2022-05-06 12:58 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin21.4.0)
  CPU: Intel(R) Core(TM) i7-6920HQ CPU @ 2.90GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, skylake)
Environment:
  JULIA_EDITOR = code
  JULIA_NUM_THREADS = 4


In [2]:
# load packages in environment
import Pkg
Pkg.activate("../../Docker")
Pkg.status()

  Activating project at `~/Documents/github.com/JSM-Julia-Short-Course-2022/Docker`


      Status `~/Documents/github.com/JSM-Julia-Short-Course-2022/Docker/Project.toml`
  [1520ce14] AbstractTrees v0.3.4
  [336ed68f] CSV v0.10.4
  [31c24e10] Distributions v0.25.66
  [38e38edf] GLM v1.8.0
  [28b8d3ca] GR v0.66.0
  [c27321d9] Glob v1.3.0
  [bdcacae8] LoopVectorization v0.12.121
  [a15396b6] OnlineStats v1.5.13
  [ca7969ec] PlotlyLight v0.5.1
  [91a5bcdd] Plots v1.31.5
  [c3e4b0f8] Pluto v0.19.11
  [7f904dfe] PlutoUI v0.7.39
  [438e738f] PyCall v1.93.1
  [6f49c342] RCall v0.13.13
  [ce6b1742] RDatasets v0.7.7
  [2913bbd2] StatsBase v0.33.20
  [f3b207a7] StatsPlots v0.15.1
  [b8865327] UnicodePlots v3.0.4


## Introduction

* Topics in numerical algebra: 
    - BLAS  
    - solve linear equations $\mathbf{A} \mathbf{x} = \mathbf{b}$
    - regression computations $\mathbf{X}^T \mathbf{X} \beta = \mathbf{X}^T \mathbf{y}$  
    - eigen-problems $\mathbf{A} \mathbf{x} = \lambda \mathbf{x}$  
    - generalized eigen-problems $\mathbf{A} \mathbf{x} = \lambda \mathbf{B} \mathbf{x}$  
    - singular value decompositions $\mathbf{A} = \mathbf{U} \Sigma \mathbf{V}^T$  
    - iterative methods for numerical linear algebra    

* Except for the iterative methods, most of these numerical linear algebra tasks are implemented in the BLAS and LAPACK libraries. They form the **building blocks** of most statistical computing tasks (optimization, MCMC).

* Our major **goal** (or learning objectives) is to  
    1. know the complexity (flop count) of each task
    2. be familiar with the BLAS and LAPACK functions (what they do)  
    3. do **not** re-invent wheels by implementing these dense linear algebra subroutines by yourself  
    4. understand the need for iterative methods  
    5. apply appropriate numerical algebra tools to various statistical problems 

* All high-level languages (R, Matlab, Julia) call BLAS and LAPACK for numerical linear algebra. 
    - Julia offers more flexibility by exposing interfaces to many BLAS/LAPACK subroutines directly. See [documentation](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#BLAS-functions-1).

## BLAS

* BLAS stands for _basic linear algebra subprograms_. 

* See [netlib](http://www.netlib.org/blas/) for a complete list of standardized BLAS functions.

* There are many implementations of BLAS. 
    - [Netlib](http://www.netlib.org/blas/) provides a reference implementation.  
    - Matlab uses Intel's [MKL](https://software.intel.com/en-us/node/520724) (mathematical kernel libaries). **MKL implementation is the gold standard on market.** It is not open source but the compiled library is free for Linux and MacOS.    
    - Julia uses [OpenBLAS](https://github.com/xianyi/OpenBLAS). **OpenBLAS is the best open source implementation**.    

* There are 3 levels of BLAS functions.
    - [Level 1](http://www.netlib.org/blas/#_level_1): vector-vector operation
    - [Level 2](http://www.netlib.org/blas/#_level_2): matrix-vector operation
    - [Level 3](http://www.netlib.org/blas/#_level_3): matrix-matrix operation

| Level | Example Operation                      | Name        | Dimension                                 | Flops |  
|-------|----------------------------------------|-------------|-------------------------------------------|-------|
| 1     | $\alpha \gets \mathbf{x}^T \mathbf{y}$ | dot product | $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$ | $2n$  |  
| 1 | $\mathbf{y} \gets \mathbf{y} + \alpha \mathbf{x}$ |  axpy           |  $\alpha \in \mathbb{R}$, $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$ |  $2n$    |  
| 2     | $\mathbf{y} \gets \mathbf{y} + \mathbf{A} \mathbf{x}$ |  gaxpy           |  $\mathbf{A} \in \mathbb{R}^{m \times n}$, $\mathbf{x} \in \mathbb{R}^n$, $\mathbf{y} \in \mathbb{R}^m$                                     |  $2mn$     |
| 2 | $\mathbf{A} \gets \mathbf{A} + \mathbf{y} \mathbf{x}^T$ | rank one update            |    $\mathbf{A} \in \mathbb{R}^{m \times n}$, $\mathbf{x} \in \mathbb{R}^n$, $\mathbf{y} \in \mathbb{R}^m$                                       | $2mn$      |
| 3     | $\mathbf{C} \gets \mathbf{C} + \mathbf{A} \mathbf{B}$                                       |  matrix multiplication           |  $\mathbf{A} \in \mathbb{R}^{m \times p}$, $\mathbf{B} \in \mathbb{R}^{p \times n}$, $\mathbf{C} \in \mathbb{R}^{m \times n}$                                         | $2mnp$      |

* Typical BLAS functions support single precision (S), double precision (D), complex (C), and double complex (Z). 

## Examples

> **The form of a mathematical expression and the way the expression should be evaluated in actual practice may be quite different.**

Some operations _appear_ as level-3 but indeed are level-2.  

**Example 1**. A common operation in statistics is column scaling or row scaling
$$
\begin{eqnarray*}
    \mathbf{A} &=& \mathbf{A} \mathbf{D} \quad \text{(column scaling)} \\
    \mathbf{A} &=& \mathbf{D} \mathbf{A} \quad \text{(row scaling)},
\end{eqnarray*}
$$
where $\mathbf{D}$ is diagonal. For example, in generalized linear models (GLMs), the Fisher information matrix takes the form  
$$
\mathbf{X}^T \mathbf{W} \mathbf{X},
$$
where $\mathbf{W}$ is a diagonal matrix with observation weights on diagonal.  

  Column and row scalings are essentially level-2 operations!

In [3]:
using BenchmarkTools, LinearAlgebra, Random

Random.seed!(123) # seed
n = 2000
A = rand(n, n) # n-by-n matrix
d = rand(n)  # n vector
D = Diagonal(d) # diagonal matrix with d as diagonal

2000×2000 Diagonal{Float64, Vector{Float64}}:
 0.919181   ⋅         ⋅         ⋅        …   ⋅         ⋅          ⋅ 
  ⋅        0.426019   ⋅         ⋅            ⋅         ⋅          ⋅ 
  ⋅         ⋅        0.746586   ⋅            ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅        0.819201      ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅            ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅        …   ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅            ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅            ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅            ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅            ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅        …   ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅            ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅            ⋅         ⋅          ⋅ 
 ⋮                                       ⋱               

In [4]:
Dfull = convert(Matrix, D) # convert to full matrix

2000×2000 Matrix{Float64}:
 0.919181  0.0       0.0       0.0       …  0.0       0.0        0.0
 0.0       0.426019  0.0       0.0          0.0       0.0        0.0
 0.0       0.0       0.746586  0.0          0.0       0.0        0.0
 0.0       0.0       0.0       0.819201     0.0       0.0        0.0
 0.0       0.0       0.0       0.0          0.0       0.0        0.0
 0.0       0.0       0.0       0.0       …  0.0       0.0        0.0
 0.0       0.0       0.0       0.0          0.0       0.0        0.0
 0.0       0.0       0.0       0.0          0.0       0.0        0.0
 0.0       0.0       0.0       0.0          0.0       0.0        0.0
 0.0       0.0       0.0       0.0          0.0       0.0        0.0
 0.0       0.0       0.0       0.0       …  0.0       0.0        0.0
 0.0       0.0       0.0       0.0          0.0       0.0        0.0
 0.0       0.0       0.0       0.0          0.0       0.0        0.0
 ⋮                                       ⋱                       
 0.0      

In [5]:
# this is calling BLAS routine for matrix multiplication: O(n^3) flops
# this is SLOW!
@benchmark $A * $Dfull

BenchmarkTools.Trial: 47 samples with 1 evaluation.
 Range (min … max):   99.118 ms … 132.413 ms  ┊ GC (min … max): 0.00% … 5.46%
 Time  (median):     106.088 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   107.356 ms ±   7.122 ms  ┊ GC (mean ± σ):  2.11% ± 2.95%

  █ ▃█ █▃█  █ ▃▃    ▃█   █ ▃                                     
  █▇██▇███▇▇█▁██▇▇▁▁██▁▇▇█▁█▇▇▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▇▁▁▁▁▁▁▁▁▇ ▁
  99.1 ms          Histogram: frequency by time          132 ms <

 Memory estimate: 30.52 MiB, allocs estimate: 2.

In [6]:
# dispatch to special method for diagonal matrix multiplication.
# columnwise scaling: O(n^2) flops
@benchmark $A * $D

BenchmarkTools.Trial: 405 samples with 1 evaluation.
 Range (min … max):   9.118 ms … 67.305 ms  ┊ GC (min … max):  0.00% …  0.00%
 Time  (median):      9.720 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   12.324 ms ±  4.643 ms  ┊ GC (mean ± σ):  19.26% ± 19.82%

  ▅█▇▄▂     ▁                           ▅▅▃▂                   
  ███████▄▆▇█▇▁▁▄▄▁▁▁▁▁▁▁▁▁▁▁▁▄▄▁▁▁▁▁▁▁▁████▇▇█▆▄▅▄▄▄▄▁▄▁▁▄▁▅ ▇
  9.12 ms      Histogram: log(frequency) by time      20.2 ms <

 Memory estimate: 30.52 MiB, allocs estimate: 4.

In [7]:
# in-place: avoid allocate space for result
# rmul!: compute matrix-matrix product AB, overwriting A, and return the result.
@benchmark rmul!($A, $D)

BenchmarkTools.Trial: 459 samples with 1 evaluation.
 Range (min … max):   5.224 ms … 31.493 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      9.435 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.878 ms ±  5.214 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

   █▇                                                          
  ████▃▃▄▃▃▅▅▃▄▃▄▂▅▅▃▃▂▃▄▄▇▇▄▅▅▃▂▁▃▃▃▃▃▅▇▃▃▂▂▁▁▁▁▂▂▁▁▁▁▂▁▂▁▁▂ ▃
  5.22 ms         Histogram: frequency by time        25.6 ms <

 Memory estimate: 96 bytes, allocs estimate: 2.

**Note:** In R or Matlab, `diag(d)` will create a full matrix. Be cautious using `diag` function: do we really need a full diagonal matrix?

In [8]:
using RCall

R"""
d <- runif(5)
diag(d)
"""

┌ Info: Precompiling RCall [6f49c342-dc21-5d91-9882-a32aef131414]
└ @ Base loading.jl:1423


RObject{RealSxp}
          [,1]       [,2]      [,3]      [,4]      [,5]
[1,] 0.6350053 0.00000000 0.0000000 0.0000000 0.0000000
[2,] 0.0000000 0.03440707 0.0000000 0.0000000 0.0000000
[3,] 0.0000000 0.00000000 0.2422235 0.0000000 0.0000000
[4,] 0.0000000 0.00000000 0.0000000 0.9490331 0.0000000
[5,] 0.0000000 0.00000000 0.0000000 0.0000000 0.6110186


**Example 2**. Innter product between two matrices $\mathbf{A}, \mathbf{B} \in \mathbb{R}^{m \times n}$ is often written as 
$$
    \text{trace}(\mathbf{A}^T \mathbf{B}), \text{trace}(\mathbf{B} \mathbf{A}^T), \text{trace}(\mathbf{A} \mathbf{B}^T), \text{ or } \text{trace}(\mathbf{B}^T \mathbf{A}).
$$
They appear as level-3 operation (matrix multiplication with $O(m^2n)$ or $O(mn^2)$ flops).

In [9]:
Random.seed!(123)
n = 2000
A, B = randn(n, n), randn(n, n)

# slow way to evaluate this thing
@benchmark tr(transpose($A) * $B)

BenchmarkTools.Trial: 46 samples with 1 evaluation.
 Range (min … max):  100.275 ms … 134.056 ms  ┊ GC (min … max): 0.00% … 7.87%
 Time  (median):     107.500 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   109.992 ms ±   8.246 ms  ┊ GC (mean ± σ):  2.22% ± 3.55%

   ▄ ▄ ▄ █  ▁  █  ▁▁       ▁    ▁            ▁                   
  ▆█▁█▆█▆█▁▆█▆▆█▆▆██▆▆▁▆▁▁▁█▁▁▁▁█▁▆▁▁▁▁▁▆▁▆▁▁█▁▁▆▁▁▁▁▆▁▁▁▁▁▁▁▁▆ ▁
  100 ms           Histogram: frequency by time          134 ms <

 Memory estimate: 30.52 MiB, allocs estimate: 2.

But $\text{trace}(\mathbf{A}^T \mathbf{B}) = <\text{vec}(\mathbf{A}), \text{vec}(\mathbf{B})>$. The latter is level-2 operation with $O(mn)$ flops.

In [10]:
@benchmark dot($A, $B)

BenchmarkTools.Trial: 1405 samples with 1 evaluation.
 Range (min … max):  2.517 ms …   7.797 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.479 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.530 ms ± 396.951 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                      ▁▁▁▃▇█▃▅▃                                
  ▂▂▂▂▃▁▂▂▃▃▄▄▄▄▅▆▆▇█▇███████████▆█▅▅▅▅▄▄▄▄▄▃▃▄▄▃▃▃▂▃▂▃▃▂▂▂▃▂ ▄
  2.52 ms         Histogram: frequency by time        4.74 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

**Example 3**. Similarly $\text{diag}(\mathbf{A}^T \mathbf{B})$ can be calculated in $O(mn)$ flops.

In [11]:
# slow way to evaluate this thing: O(n^3)
@benchmark diag(transpose($A) * $B)

BenchmarkTools.Trial: 46 samples with 1 evaluation.
 Range (min … max):  100.188 ms … 142.473 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     109.716 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   111.167 ms ±   8.859 ms  ┊ GC (mean ± σ):  2.12% ± 3.44%

  ▁  █▃                     ▃                                    
  █▄▁██▄▁▁▄▇▄▇▇▁▇▇▁▁▄▁▇▇▁▇▄▄█▄▁▄▁▁▁▄▁▄▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  100 ms           Histogram: frequency by time          142 ms <

 Memory estimate: 30.53 MiB, allocs estimate: 3.

In [12]:
# smarter: O(n^2)
@benchmark Diagonal(vec(sum($A .* $B, dims=1)))

BenchmarkTools.Trial: 432 samples with 1 evaluation.
 Range (min … max):   9.072 ms … 19.894 ms  ┊ GC (min … max):  0.00% … 52.51%
 Time  (median):      9.213 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   11.559 ms ±  4.065 ms  ┊ GC (mean ± σ):  20.15% ± 21.74%

  █▄                                                     ▄▂    
  ███▆▇▄▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▅███▇ ▆
  9.07 ms      Histogram: log(frequency) by time      19.1 ms <

 Memory estimate: 30.53 MiB, allocs estimate: 5.

To get rid of allocation of intermediate array at all, we can just write a double loop or use `dot` function.

In [13]:
using LoopVectorization

function diag_matmul!(d, A, B)
    m, n = size(A)
    @assert size(B) == (m, n) "A and B should have same size"
    fill!(d, 0)
    @avx for j in 1:n, i in 1:m
        d[j] += A[i, j] * B[i, j]
    end
#     for j in 1:n
#         @views d[j] = dot(A[:, j], B[:, j])
#     end
    Diagonal(d)
end

d = zeros(eltype(A), size(A, 2))
@benchmark diag_matmul!($d, $A, $B)

┌ Info: Precompiling LoopVectorization [bdcacae8-1622-11e9-2a5c-532679323890]
└ @ Base loading.jl:1423


BenchmarkTools.Trial: 1404 samples with 1 evaluation.
 Range (min … max):  3.234 ms …   4.547 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.505 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.546 ms ± 153.563 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

              ▂▅▇█▇▅▃▄▂                                        
  ▂▁▁▂▂▃▃▃▃▃▅██████████▇▆▆▄▅▅▄▃▃▃▃▃▃▃▃▃▂▂▃▂▃▂▂▂▃▂▃▂▂▂▂▂▂▃▃▂▂▂ ▄
  3.23 ms         Histogram: frequency by time        4.16 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

## Memory hierarchy and level-3 fraction

> **Key to high performance is effective use of memory hierarchy. True on all architectures.**

* Flop count is not the sole determinant of algorithm efficiency. Another important factor is data movement through the memory hierarchy.

<img src="./macpro_inside.png" width="400" align="center">

<img src="./cpu_die.png" width="400" align="center">  

<img src="./hei.png" width="400" align="center">

* Numbers everyone should know

| Operation                           | Time           |
|-------------------------------------|----------------|
| L1 cache reference                  | 0.5 ns         |
| L2 cache reference                  | 7 ns           |
| Main memory reference               | 100 ns         |
| Read 1 MB sequentially from memory  | 250,000 ns     |
| Read 1 MB sequentially from SSD     | 1,000,000 ns   |  
| Read 1 MB sequentially from disk    | 20,000,000 ns  |


<!-- | Operation                           | Time           | -->
<!-- |-------------------------------------|----------------| -->
<!-- | L1 cache reference                  | 0.5 ns         | -->
<!-- | Branch mispredict                   | 5 ns           | -->
<!-- | L2 cache reference                  | 7 ns           | -->
<!-- | Mutex lock/unlock                   | 100 ns         | -->
<!-- | Main memory reference               | 100 ns         | -->
<!-- | Compress 1K bytes with Zippy        | 10,000 ns      | -->
<!-- | Send 2K bytes over 1 Gbps network   | 20,000 ns      | -->
<!-- | Read 1 MB sequentially from memory  | 250,000 ns     | -->
<!-- | Round trip within same datacenter   | 500,000 ns     | -->
<!-- | Disk seek                           | 10,000,000 ns  | -->
<!-- | Read 1 MB sequentially from network | 10,000,000 ns  | -->
<!-- | Read 1 MB sequentially from disk    | 30,000,000 ns  | -->
<!-- | Send packet CA->Netherlands->CA     | 150,000,000 ns | -->

   Source: <https://gist.github.com/jboner/2841832>  

* For example, Xeon X5650 CPU has a theoretical throughput of 128 DP GFLOPS but a max memory bandwidth of 32GB/s.  

* Can we keep CPU cores busy with enough deliveries of matrix data and ship the results to memory fast enough to avoid backlog?  
Answer: use **high-level BLAS** as much as possible.

| BLAS | Dimension | Mem. Refs. | Flops  | Ratio |
|--------------------------------|------------------------------------------------------------|------------|--------|-------|
| Level 1: $\mathbf{y} \gets \mathbf{y} + \alpha \mathbf{x}$     | $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$                                           | $3n$       | $2n$   | 3:2   |
| Level 2: $\mathbf{y} \gets \mathbf{y} + \mathbf{A} \mathbf{x}$ | $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$, $\mathbf{A} \in \mathbb{R}^{n \times n}$ | $n^2$      | $2n^2$ | 1:2   |
| Level 3: $\mathbf{C} \gets \mathbf{C} + \mathbf{A} \mathbf{B}$ | $\mathbf{A}, \mathbf{B}, \mathbf{C} \in\mathbb{R}^{n \times n}$                    | $4n^2$     | $2n^3$ | 2:n |  

* Higher level BLAS (3 or 2) make more effective use of arithmetic logic units (ALU) by keeping them busy. **Surface-to-volume** effect.  
See [Dongarra slides](https://www.samsi.info/wp-content/uploads/2017/02/SAMSI-0217_Dongarra.pdf).

<img src="./blas_throughput.png" width="500" align="center"/>

* A distinction between LAPACK and LINPACK (older version of R uses LINPACK) is that LAPACK makes use of higher level BLAS as much as possible (usually by smart partitioning) to increase the so-called **level-3 fraction**.

* To appreciate the efforts in an optimized BLAS implementation such as OpenBLAS (evolved from GotoBLAS), see the [Quora question](https://www.quora.com/What-algorithm-does-BLAS-use-for-matrix-multiplication-Of-all-the-considerations-e-g-cache-popular-instruction-sets-Big-O-etc-which-one-turned-out-to-be-the-primary-bottleneck), especially the [video](https://youtu.be/JzNpKDW07rw). Bottomline is 

> **Get familiar with (good implementations of) BLAS/LAPACK and use them as much as possible.**

## Effect of data layout

* Data layout in memory affects algorithmic efficiency too. It is much faster to move chunks of data in memory than retrieving/writing scattered data.

* Storage mode: **column-major** (Fortran, Matlab, R, Julia) vs **row-major** (C/C++).

* **Cache line** is the minimum amount of cache which can be loaded and stored to memory.
    - x86 CPUs: 64 bytes  
    - ARM CPUs: 32 bytes

<img src="https://patterns.eecs.berkeley.edu/wordpress/wp-content/uploads/2013/04/dense02.png" width="500" align="center"/>

* Accessing column-major stored matrix by rows ($ij$ looping) causes lots of **cache misses**.

* Take matrix multiplication as an example 
$$ 
\mathbf{C} \gets \mathbf{C} + \mathbf{A} \mathbf{B}, \quad \mathbf{A} \in \mathbb{R}^{m \times p}, \mathbf{B} \in \mathbb{R}^{p \times n}, \mathbf{C} \in \mathbb{R}^{m \times n}.
$$
Assume the storage is column-major, such as in Julia. There are 6 variants of the algorithms according to the order in the triple loops. 
    - `jki` or `kji` looping:
        ```julia
        # inner most loop
        for i = 1:m
            C[i, j] = C[i, j] + A[i, k] * B[k, j]
        end
        ```  
    - `ikj` or `kij` looping:
        ```julia
        # inner most loop        
        for j = 1:n
            C[i, j] = C[i, j] + A[i, k] * B[k, j]
        end
        ```  
    - `ijk` or `jik` looping:
        ```julia
        # inner most loop        
        for k = 1:p
            C[i, j] = C[i, j] + A[i, k] * B[k, j]
        end
        ```
* We pay attention to the innermost loop, where the vector calculation occurs. The associated **stride** when accessing the three matrices in memory (assuming column-major storage) is  

| Variant        | A Stride | B Stride | C Stride |
|----------------|----------|----------|----------|
| $jki$ or $kji$ | Unit     | 0        | Unit     |
| $ikj$ or $kij$ | 0        | Non-Unit | Non-Unit |
| $ijk$ or $jik$ | Non-Unit | Unit     | 0        |       
Apparently the variants $jki$ or $kji$ are preferred.

In [14]:
"""
    matmul_by_loop!(A, B, C, order)

Overwrite `C` by `A * B`. `order` indicates the looping order for triple loop.
"""
function matmul_by_loop!(A::Matrix, B::Matrix, C::Matrix, order::String)
    
    m = size(A, 1)
    p = size(A, 2)
    n = size(B, 2)
    fill!(C, 0)
    
    if order == "jki"
        @inbounds for j = 1:n, k = 1:p, i = 1:m
            C[i, j] += A[i, k] * B[k, j]
        end
    end

    if order == "kji"
        @inbounds for k = 1:p, j = 1:n, i = 1:m
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
    if order == "ikj"
        @inbounds for i = 1:m, k = 1:p, j = 1:n
            C[i, j] += A[i, k] * B[k, j]
        end
    end

    if order == "kij"
        @inbounds for k = 1:p, i = 1:m, j = 1:n
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
    if order == "ijk"
        @inbounds for i = 1:m, j = 1:n, k = 1:p
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
    if order == "jik"
        @inbounds for j = 1:n, i = 1:m, k = 1:p
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
end

using Random

Random.seed!(123)
m, p, n = 2000, 100, 2000
A = rand(m, p)
B = rand(p, n)
C = zeros(m, n);

* $jki$ and $kji$ looping:

In [15]:
using BenchmarkTools

@benchmark matmul_by_loop!($A, $B, $C, "jki")

BenchmarkTools.Trial: 65 samples with 1 evaluation.
 Range (min … max):  71.276 ms … 87.572 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     76.309 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   76.900 ms ±  3.370 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

                      █                                        
  ▄▁▆▄▁▁▆▄▄▆█▁▆▄▁▆▁▄▄▁█▆█▄▄▁█▆▁█▆▁▁▆█▆▁▁▁▄▆▄▁▁▁▆▁▄▄▄▁▁▁▄▁▄▁▁▆ ▁
  71.3 ms         Histogram: frequency by time        83.8 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [16]:
@benchmark matmul_by_loop!($A, $B, $C, "kji")

BenchmarkTools.Trial: 11 samples with 1 evaluation.
 Range (min … max):  460.260 ms … 503.676 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     479.927 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   482.656 ms ±  13.682 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁      ▁               █ ▁ ▁    ▁  ▁                  ▁▁    ▁  
  █▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁█▁█▁▁▁▁█▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██▁▁▁▁█ ▁
  460 ms           Histogram: frequency by time          504 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

* $ikj$ and $kij$ looping:

In [17]:
@benchmark matmul_by_loop!($A, $B, $C, "ikj")

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  2.676 s …   2.795 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.735 s              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.735 s ± 84.124 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.68 s         Histogram: frequency by time        2.79 s <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [18]:
@benchmark matmul_by_loop!($A, $B, $C, "kij")

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  2.798 s …   2.829 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.813 s              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.813 s ± 21.922 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.8 s          Histogram: frequency by time        2.83 s <

 Memory estimate: 0 bytes, allocs estimate: 0.

* $ijk$ and $jik$ looping:

In [19]:
@benchmark matmul_by_loop!($A, $B, $C, "ijk")

BenchmarkTools.Trial: 10 samples with 1 evaluation.
 Range (min … max):  492.677 ms … 507.581 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     502.457 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   501.144 ms ±   5.269 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █ █               █          █       █    █   █     █  █    █  
  █▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁█▁▁▁█▁▁▁▁▁█▁▁█▁▁▁▁█ ▁
  493 ms           Histogram: frequency by time          508 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [20]:
@benchmark matmul_by_loop!($A, $B, $C, "ijk")

BenchmarkTools.Trial: 10 samples with 1 evaluation.
 Range (min … max):  496.005 ms … 508.096 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     502.363 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   502.265 ms ±   4.295 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁     ▁    ▁          ▁      ▁    ▁    ▁               █    ▁  
  █▁▁▁▁▁█▁▁▁▁█▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁█▁▁▁▁█▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁█ ▁
  496 ms           Histogram: frequency by time          508 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

* Julia wraps BLAS library for matrix multiplication. We see BLAS library wins hands down (multi-threading, Strassen algorithm, higher level-3 fraction by block outer product).

In [21]:
@benchmark mul!($C, $A, $B)

BenchmarkTools.Trial: 638 samples with 1 evaluation.
 Range (min … max):  7.086 ms …  11.535 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.483 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.816 ms ± 773.185 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▁█▄▂                                                       
  ▂▂█████▇▆▆▅▄▃▃▄▃▃▃▃▃▂▃▃▃▃▃▂▃▃▃▂▂▃▂▃▂▃▃▁▂▂▁▂▂▂▂▂▂▂▂▂▂▃▂▂▂▁▂▂ ▃
  7.09 ms         Histogram: frequency by time        10.5 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [22]:
# direct call of BLAS wrapper function
@benchmark LinearAlgebra.BLAS.gemm!('N', 'N', 1.0, $A, $B, 0.0, $C)

BenchmarkTools.Trial: 637 samples with 1 evaluation.
 Range (min … max):  7.208 ms …  11.057 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.483 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.830 ms ± 761.501 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄█▆▆▅▅▃▃▂▁▂▁▁        ▁     ▁                                 
  ██████████████▇▇▇█▇▇▇█▇▇▇█▇█▇▆▇▇▅▇█▇▇▇▆▅▆▆▅▁▇▅▇▅▆▁▁▁▁▁▁▅▁▁▅ █
  7.21 ms      Histogram: log(frequency) by time      10.5 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

**Exercise:** Annotate the loop in `matmul_by_loop!` by `@avx` and benchmark again. 

## BLAS in R

* **Tip for R user**. Standard R distribution from CRAN uses a very out-dated BLAS/LAPACK library.

In [23]:
using RCall

R"""
library(dplyr)
library(bench)
bench::mark($A %*% $B) %>%
  print(width = Inf)
""";

┌ Warning: RCall.jl: 
│ Attaching package: ‘dplyr’
│ 
│ The following objects are masked from ‘package:stats’:
│ 
│     filter, lag
│ 
│ The following objects are masked from ‘package:base’:
│ 
│     intersect, setdiff, setequal, union
│ 
└ @ RCall /Users/huazhou/.julia/packages/RCall/6kphM/src/io.jl:172


# A tibble: 1 × 13
  expression               min   median `itr/sec` mem_alloc `gc/sec` n_itr  n_gc
  <bch:expr>          <bch:tm> <bch:tm>     <dbl> <bch:byt>    <dbl> <int> <dbl>
1 `#JL`$A %*% `#JL`$B    245ms    276ms      3.62    30.5MB     3.62     2     2
  total_time result                memory             time          
    <bch:tm> <list>                <list>             <list>        
1      553ms <dbl [2,000 × 2,000]> <Rprofmem [2 × 3]> <bench_tm [2]>
  gc              
  <list>          
1 <tibble [2 × 3]>


┌ Warning: RCall.jl: Warning: Some expressions had a GC in every iteration; so filtering is disabled.
└ @ RCall /Users/huazhou/.julia/packages/RCall/6kphM/src/io.jl:172


* Re-build R from source using OpenBLAS or MKL will immediately boost linear algebra performance in R. Google `build R using MKL` to get started. Similarly we can build Julia using MKL.

* Matlab uses MKL. Usually it's very hard to beat Matlab in terms of linear algebra.

## Avoid memory allocation: some examples

1. Transposing matrix is an expensive memory operation.  
    - In R, the command 
        ```R
        t(A) %*% x
        ```
    will first transpose `A` then perform matrix multiplication, causing unnecessary memory allocation
    - Julia is smart to avoid transposing matrix if possible.

In [24]:
using Random, LinearAlgebra, BenchmarkTools
Random.seed!(123)

n = 1000
A = rand(n, n)
x = rand(n);

In [25]:
typeof(transpose(A))

Transpose{Float64, Matrix{Float64}}

In [26]:
fieldnames(typeof(transpose(A)))

(:parent,)

In [27]:
# same data in tranpose(A) and original matrix A
pointer(transpose(A).parent), pointer(A)

(Ptr{Float64} @0x00007fa153e00000, Ptr{Float64} @0x00007fa153e00000)

In [28]:
# dispatch to BLAS
# does *not* actually transpose the matrix
@benchmark transpose($A) * $x

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   93.503 μs … 378.568 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     118.308 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   125.545 μs ±  21.869 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▁▃▃▄▆██▇▆▆▅▃▃▃▃▂▁▁▁ ▁▁▁▂▁▁ ▁                             ▂
  ▂▄▆▅▆█████████████████████████████▇▇▇▇▇██▇▇█▇▇▇▇█▆▇▆▅▆▅▇▆▅▆▆▆ █
  93.5 μs       Histogram: log(frequency) by time        221 μs <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

In [29]:
# pre-allocate result
out = zeros(size(A, 2))
@benchmark mul!($out, transpose($A), $x)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  104.756 μs … 850.977 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     154.806 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   159.863 μs ±  27.378 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

             ▁▂▃▄▆▇█▇▆▃▃▃▆▇▄▁                                    
  ▁▁▁▁▁▁▂▃▃▅▆████████████████▆▅▄▄▄▄▃▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  105 μs           Histogram: frequency by time          257 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [30]:
# or call BLAS wrapper directly
@benchmark LinearAlgebra.BLAS.gemv!('T', 1.0, $A, $x, 0.0, $out)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   95.031 μs … 454.884 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     133.978 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   138.288 μs ±  19.020 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                ▁▆▅▇█▆▃▁▁▁▁▁                                     
  ▁▁▁▁▁▁▁▁▂▂▂▃▅▆█████████████▇▆▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  95 μs            Histogram: frequency by time          214 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

2. [Broadcasting](https://docs.julialang.org/en/v1/base/arrays/#Broadcast-and-vectorization-1) in Julia achieves vectorized code without creating intermediate arrays.

    Suppose we want to calculate elementsize maximum of absolute values of two large arrays. In R or Matlab, the command
```r
max(abs(X), abs(Y))
```
will create two intermediate arrays and then one result array.

In [31]:
using RCall
Random.seed!(123)
X, Y = rand(1000, 1000), rand(1000, 1000)

R"""
library(dplyr)
library(bench)
bench::mark(max(abs($X), abs($Y))) %>%
  print(width = Inf)
""";

# A tibble: 1 × 13
  expression                           min   median `itr/sec` mem_alloc `gc/sec`
  <bch:expr>                      <bch:tm> <bch:tm>     <dbl> <bch:byt>    <dbl>
1 max(abs(`#JL`$X), abs(`#JL`$Y))   5.75ms   6.43ms      145.    15.3MB     68.6
  n_itr  n_gc total_time result    memory             time           
  <int> <dbl>   <bch:tm> <list>    <list>             <list>         
1    38    18      263ms <dbl [1]> <Rprofmem [2 × 3]> <bench_tm [56]>
  gc               
  <list>           
1 <tibble [56 × 3]>


In Julia, dot operations are fused so no intermediate arrays are created.

In [32]:
# no intermediate arrays created, only result array created
@benchmark max.(abs.($X), abs.($Y))

BenchmarkTools.Trial: 2095 samples with 1 evaluation.
 Range (min … max):  1.451 ms … 10.805 ms  ┊ GC (min … max):  0.00% … 82.08%
 Time  (median):     1.848 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.377 ms ±  1.831 ms  ┊ GC (mean ± σ):  23.27% ± 22.00%

    ▇█▂                                                  ▂▂▁  
  ▄▁███▇▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▆███ █
  1.45 ms      Histogram: log(frequency) by time     8.87 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

Pre-allocating result array gets rid of memory allocation at all.

In [33]:
# no memory allocation at all!
Z = zeros(size(X)) # zero matrix of same size as X
@benchmark $Z .= max.(abs.($X), abs.($Y)) # .= (vs =) is important!

BenchmarkTools.Trial: 3842 samples with 1 evaluation.
 Range (min … max):  1.131 ms …  2.480 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.264 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.291 ms ± 99.764 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

          ▂▄▆▇█▆▄▁▂                                           
  ▁▃▃▃▃▂▃▆█████████▇█▆▆▅▅▅▅▅▄▃▄▃▃▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  1.13 ms        Histogram: frequency by time        1.67 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

3. [View](https://docs.julialang.org/en/v1/base/arrays/#Views-(SubArrays-and-other-view-types)-1) avoids creating extra copy of matrix data.

In [34]:
Random.seed!(123)
A = randn(1000, 1000)

# sum entries in a sub-matrix
@benchmark sum($A[1:2:500, 1:2:500])

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   73.893 μs …   8.233 ms  ┊ GC (min … max):  0.00% … 95.18%
 Time  (median):     274.057 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   300.502 μs ± 531.197 μs  ┊ GC (mean ± σ):  12.38% ±  6.77%

                                         ▃▇██▆▃▁                 
  ▃▇▄▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁▂▂▃▅████████▇▆▄▄▃▃▃▂▂▂▂▂▂▂▂ ▃
  73.9 μs          Histogram: frequency by time          364 μs <

 Memory estimate: 488.36 KiB, allocs estimate: 2.

In [35]:
# view avoids creating a separate sub-matrix
@benchmark sum(@view $A[1:2:500, 1:2:500])

BenchmarkTools.Trial: 7165 samples with 1 evaluation.
 Range (min … max):  668.951 μs …  1.293 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     688.115 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   695.493 μs ± 56.016 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █  ▇▅▁ ▁ ▁▁▁▁ ▁▁▁                                            ▁
  ██▆███████████████████▇█▇█▇█▇▇▆▆▇▇▆▆▆▆▅▅▆▅▅▅▅▄▆▄▃▄▃▅▄▅▄▄▄▁▅▄ █
  669 μs        Histogram: log(frequency) by time       959 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

The [`@views`](https://docs.julialang.org/en/v1/base/arrays/#Base.@views) macro, which can be useful in [some operations](https://discourse.julialang.org/t/why-is-a-manual-in-place-addition-so-much-faster-than-and-on-range-indexed-arrays/3302).

In [36]:
@benchmark @views sum($A[1:2:500, 1:2:500])

BenchmarkTools.Trial: 7071 samples with 1 evaluation.
 Range (min … max):  668.961 μs …  3.128 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     688.216 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   704.517 μs ± 58.085 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆   █▄   ▄▁   ▁   ▁                                          ▁
  █▅▃▆███▇████▇█████████▇▇█▇▇▇▇▆▇▇▇▇▆▇▆▆▆▅▆▅▆▅▅▅▆▅▅▄▆▄▅▆▅▄▄▅▄▅ █
  669 μs        Histogram: log(frequency) by time       918 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.